# The very first Virtual Rat RNN

In [41]:
import numpy as np
from RNN import FirstRNN
from VirtualRatFunctions import *
from RNN_solver import RNNsolver
# auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

allRatsData = getData()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loaded defaults from ~/.dbconf
Z009 (12771, 6)
A099 (23265, 6)
A109 (42333, 6)
Z014 (37888, 6)
Z010 (55682, 6)
A110 (27157, 6)
A112 (15870, 6)
A111 (56784, 6)
Z006 (40734, 6)
A117 (20919, 6)
A119 (29117, 6)
A121 (29079, 6)
A120 (18619, 6)
A106 (92159, 6)
A098 (68163, 6)


In [44]:
rat = allRatsData["Z009"]

x = np.zeros((1, rat.shape[0], 3))
y = np.zeros((1, rat.shape[0]))
trueY = np.zeros((1, rat.shape[0]))

x[0,:,:] = rat[:,:3]

train_num = int(rat.shape[0] * 0.8)
val_num = int(rat.shape[0] * 0.9)

trainX = x[:,:train_num,:]
valX = x[:,train_num:val_num,:]
testX = x[:,val_num:,:]

y[0,rat[:,3]>0] = 0
y[0,rat[:,4]>0] = 1
y[0,rat[:,5]>0] = 2

trainY = y[:,:train_num]
valY = y[:,train_num:val_num]
testY = y[:,val_num:]

trueY[0,:] = np.logical_not(np.bitwise_xor(rat[:,0],rat[:,1]))
trainTrueY = trueY[:,:train_num]
valTrueY = trueY[:,train_num:val_num]
testTrueY = trueY[:,val_num:]

In [55]:
RNN = FirstRNN()
solver = RNNsolver(RNN, trainX, trainY,optim_config={
             'learning_rate': 1e-6,
           })
solver.train()

(Iteration 1 / 100) loss: 11978.620639
(Iteration 2 / 100) loss: 11886.505695
(Iteration 3 / 100) loss: 11812.239018
(Iteration 4 / 100) loss: 11751.622762
(Iteration 5 / 100) loss: 11701.443545
(Iteration 6 / 100) loss: 11659.266663
(Iteration 7 / 100) loss: 11623.258311
(Iteration 8 / 100) loss: 11592.040483
(Iteration 9 / 100) loss: 11564.577542
(Iteration 10 / 100) loss: 11540.089133
(Iteration 11 / 100) loss: 11517.984870
(Iteration 12 / 100) loss: 11497.815584
(Iteration 13 / 100) loss: 11479.237281
(Iteration 14 / 100) loss: 11461.984554
(Iteration 15 / 100) loss: 11445.850757
(Iteration 16 / 100) loss: 11430.673644
(Iteration 17 / 100) loss: 11416.324580
(Iteration 18 / 100) loss: 11402.700493
(Iteration 19 / 100) loss: 11389.717949
(Iteration 20 / 100) loss: 11377.308670
(Iteration 21 / 100) loss: 11365.416199
(Iteration 22 / 100) loss: 11353.993264
(Iteration 23 / 100) loss: 11342.999958
(Iteration 24 / 100) loss: 11332.402170
(Iteration 25 / 100) loss: 11322.170442
(Iteratio

In [56]:
out = RNN.predict(valX)
acc = np.mean(out == valTrueY)
print acc

0.549725920125
